In [1]:
using DataFrames
using DataArrays
using Plots
pyplot()

Plots.PyPlotBackend()

# Description of DFM

## Static Form of the DFM
Equation (6) 
$$X_t=\Lambda F_t+e_t$$
Equation (7)
$$F_t=\Phi(L)F_{t-1}+G\eta_t$$

### Principal Components Normalization
Equation (11) in the paper
$$\frac{1}{N}\Lambda'\Lambda=I_r$$ and $\Sigma_F\equiv E\left[F_tF_t^\prime\right]$ diagonal.

#### Principal Components Estimation
Equation (13)
$$\min_{F_1,\dots,F_T,\Lambda}V_r(\Lambda,F)$$
where
$$V_r\left(\Lambda,F\right)=\frac{1}{NT}\sum_{t=1}^T\left(X_t-\Lambda F_t\right)'\left(X_t-\Lambda F_t\right)$$

#### Principal Components Estimation with Missing Data
Equation (15): 
$$\min_{F_1,\dots,F_T,\Lambda}V_r\left(\Lambda,F\right)=\frac{1}{NT}\sum_{i=1}^N\sum_{t=1}^TS_{it}\left(X_t-\Lambda F_t\right)^2.$$
The objective function is minimized by iterations as follows:
1. By principal component estimation using a subset of the series for which there are no missing observations, obtain $F_t$ give $\Lambda$. 
3. Minimize over $\Lambda$ given $F_t$.
2. Minimize over $F_t$ given $\Lambda$.
4. Iterate 2 and 3 until converges.

# Figure 1
Figure 1 assumes there is only one factor which evoles as
$$f_t = \Psi_4f_{t-1=}+\dots+\Psi_4f_{t-4}.$$
All observed variables $y_t$ depend on the current value of $f$ and from first to fourth lags of $f$, and constant:
$$X_{it} = \lambda_i+\lambda_{i0}f_t+\lambda_{i1}f_{t1}+\dots+\lambda_{i4}f_{t4}.$$

In [2]:
nt_min = 20 # min number of obs for any series used to est factors

20

#### Useful functions

In [3]:
nanf(f::Function, x) = f(filter(!isnan, x))
nanmean(x) = nanf(mean, x)
nanstd(x) = nanf(std, x)
nansum(x) = nanf(sum, x)

nansum (generic function with 1 method)

In [4]:
apply_f_vertically(f::Function, df::DataFrame) =
    Matrix(map(f, eachcol(df)))
nanmean(df::DataFrame) = apply_f_vertically(nanmean, df)
nanstd(df::DataFrame) = apply_f_vertically(nanstd, df)
nansum(df::DataFrame) = apply_f_vertically(nansum, df)

nansum (generic function with 2 methods)

In [5]:
function dropna_row(df::DataFrame)
    numrows = completecases(df)
    return df[numrows, :], numrows
end
function dropna_col(df::DataFrame) 
    df_dropped, numcolt= dropna_row(df')
    return df_dropped', numcolt'
end

dropna_col (generic function with 1 method)

In [6]:
"""
##### Outputs
- `numrow`: Vector of Bool where false indicates the row has `NaN` data.
"""
function dropna_row(A::Matrix)
    numrow = .!vec(any(isnan.(A), 2))
    return A[numrow, :], numrow
end
"""
##### Outputs
- `numcolt'`: RowVector of Bool where false indicates the column has `NaN` data.
"""
function dropna_col(A::Matrix)
    At_dropped, numcolt = dropna_row(A')
    return At_dropped', numcolt'
end

dropna_col

In [7]:
function pca_score(X, nfac_u)
    _, _, V = svd(X)
    score = (X*V)[:, 1:nfac_u]
    return score
end

pca_score (generic function with 1 method)

In [8]:
"""
##### Arguments
- `y::AbstractVector`: length `T` Vector or `TxN` Matrix where `T`
                       is sample size and `N` is the number of 
                       regressed variables
- `X::AbstractArray`: `TxK` Matrix where `K` is the number of
                      regressors
##### Outputs
- `b`: OLS estimator of the coefficients
- `e`: residual
"""
function ols(y::AbstractArray, X::AbstractArray)
    b = X\y
    e = y - X*b
    return b, e
end

"""
do OLS ignoring samples containing NaN
##### Outputs
- `b`: OLS estimator of the coefficients
- `e`: residual
- `numrow`: BitArray inidicating rows used to estimate 
"""
function nanols(y::AbstractVector, X::AbstractArray)
    N = size(y, 2)
    tmp, numrow = dropna_row([y X])
    y_used, x_used = tmp[:, 1:N], tmp[:, N+1:end]
    b, e = ols(y_used, x_used)
    return b, e, numrow
end
"""
##### Arguments
- `y::AbstractMatrix`: `TxN`
- `X::AbstractArray`: `TxK` Matrix or `T` Vector
"""
function nanols(y::AbstractMatrix, X::AbstractArray)
    if size(y, 1) != size(X, 1)
        error("Sample size must be same")
    end
    T, N = size(y)
    b = Matrix{Float64}(size(X, 2), N)
    e = fill(NaN, T, N) 
    numrow = BitArray(T, N)
    for i=1:N
        tmp = nanols(y[:, i], X)
        b[:, i] = tmp[1]
        e[tmp[3], i] = tmp[2]
        numrow[:, i] = tmp[3]
    end
    return b, e, numrow
end


nanols

In [9]:
function lagmat(X::Array, lags::AbstractVector)
    nc = size(X, 2)
    Xlag = fill(NaN, size(X, 1), nc*length(lags))
    for (i, lag) in enumerate(lags)
        Xlag[lag+1:end, nc*(i-1)+1:nc*i] .= X[1:end-lag, :]
    end
    return Xlag
end
lagmat(X::Array, lag::Integer) = lagmat(X, [lag])

lagmat (generic function with 2 methods)

In [10]:
"""
`x::Array`: `nxk` matrix or vector of length `n` 
`xlag::Matrix`: `nxk` matrix
"""
function lag(x::Array, p::Integer)
    if p >= 0
        return vcat(fill(NaN, p, size(x, 2)), x[1:end-p, :])
    else
        return vcat(x[1-p:end, :], fill(NaN, -p, size(x, 2)))
    end
end

lag

In [11]:
function uar(y::Array, n_lags::Integer)
    x = Matrix{Float64}(size(y, 1), size(y, 2)*n_lags)
    for i = 1:n_lags
        x[:, size(y, 2)*(i-1)+1:size(y, 2)*i] = lag(y, i)
    end
    bols, ehat, _ = nanols(y, x)
    ssr = dot(ehat, ehat)
    ser = sqrt(ssr/(size(x, 1)-size(x, 2)))
    arcoef = bols
    return arcoef, ser
end

uar (generic function with 1 method)

## Import data

In [12]:
data = readcsv("data/bpdata.csv")
header = data[1, :]
year = data[2:end, 1]
data = Float64.(data[2:end, 2:end])

224×86 Array{Float64,2}:
 NaN            NaN            NaN            …  12.3235     NaN          
   0.0142268      0.00530395     0.0228269        9.45272      0.0152701  
  -0.0116989      0.000417996    0.0050264       -4.38442      0.00203857 
  -0.00581949    -0.00882744    -0.0629256       -2.85467     -0.00647572 
   0.012316      -0.000459845    0.0175352       -0.758017     0.0188318  
  -0.0134915      0.00268641     0.0084729    …  -8.42778     -0.00129345 
  -0.00717594    -0.0139067     -0.0219165       -6.79728     -0.00454431 
  -0.0218157     -0.00861225    -0.0393015       -8.76649     -0.0154271  
  -0.00283434    -0.0101246     -0.0704661       -6.96877     -0.0272235  
   0.00883281     0.00493428     0.0066869        8.19584     -0.0158283  
   0.00697616    -0.00494625     0.0103326    …   8.36063     -0.00347905 
   0.0105292      0.0100276      0.0335589       11.6922       0.000487977
   0.00833999     0.000812098    0.00756745       3.22391      0.0073634  


In [13]:
tmp =DataArray(data)

224×86 DataArrays.DataArray{Float64,2}:
 NaN            NaN            NaN            …  12.3235     NaN          
   0.0142268      0.00530395     0.0228269        9.45272      0.0152701  
  -0.0116989      0.000417996    0.0050264       -4.38442      0.00203857 
  -0.00581949    -0.00882744    -0.0629256       -2.85467     -0.00647572 
   0.012316      -0.000459845    0.0175352       -0.758017     0.0188318  
  -0.0134915      0.00268641     0.0084729    …  -8.42778     -0.00129345 
  -0.00717594    -0.0139067     -0.0219165       -6.79728     -0.00454431 
  -0.0218157     -0.00861225    -0.0393015       -8.76649     -0.0154271  
  -0.00283434    -0.0101246     -0.0704661       -6.96877     -0.0272235  
   0.00883281     0.00493428     0.0066869        8.19584     -0.0158283  
   0.00697616    -0.00494625     0.0103326    …   8.36063     -0.00347905 
   0.0105292      0.0100276      0.0335589       11.6922       0.000487977
   0.00833999     0.000812098    0.00756745       3.22391   

In [14]:
collect(skipmissing(tmp))

19264-element Array{Float64,1}:
 NaN          
   0.0142268  
  -0.0116989  
  -0.00581949 
   0.012316   
  -0.0134915  
  -0.00717594 
  -0.0218157  
  -0.00283434 
   0.00883281 
   0.00697616 
   0.0105292  
   0.00833999 
   ⋮          
   0.00344957 
   0.00381082 
   0.00817978 
  -0.00108383 
   0.00298895 
  -0.000650257
   0.00210145 
   0.00905227 
   0.00391703 
   0.00469112 
   0.00391343 
 NaN          

## Restrict sample

In [15]:
initperiod = 3 # first two observations are dropped

# exclude year data
xdata = Float64.(data[initperiod:end, :])

222×86 Array{Float64,2}:
 -0.0116989     0.000417996   0.0050264    …  -4.38442      0.00203857 
 -0.00581949   -0.00882744   -0.0629256       -2.85467     -0.00647572 
  0.012316     -0.000459845   0.0175352       -0.758017     0.0188318  
 -0.0134915     0.00268641    0.0084729       -8.42778     -0.00129345 
 -0.00717594   -0.0139067    -0.0219165       -6.79728     -0.00454431 
 -0.0218157    -0.00861225   -0.0393015    …  -8.76649     -0.0154271  
 -0.00283434   -0.0101246    -0.0704661       -6.96877     -0.0272235  
  0.00883281    0.00493428    0.0066869        8.19584     -0.0158283  
  0.00697616   -0.00494625    0.0103326        8.36063     -0.00347905 
  0.0105292     0.0100276     0.0335589       11.6922       0.000487977
  0.00833999    0.000812098   0.00756745   …   3.22391      0.0073634  
  0.0012303     0.0024063     0.0127609       -2.711        0.00208582 
  1.74e-5      -0.0016512    -0.00655048      -4.1126       0.00443766 
  ⋮                                    

## Standardize data

Use $\sqrt{\frac{1}{N}\sum_i^N x_i-\overline{x}}$ instead of $\sqrt{\frac{1}{N-1}\sum_i^N x_i-\overline{x}}$

In [16]:
# preprocess data to have unit standard error
xdata_df = DataFrame(xdata)

xdatamean = nanmean(xdata_df)
# make correction (which I don't understand why being needed)
tmp = size(xdata, 1) - sum(isnan.(xdata), 1)
tmp = (tmp-1)./tmp
xdatastd = nanstd(xdata_df).*sqrt.(tmp)

data_standardized  = (xdata .- xdatamean)./xdatastd

222×86 Array{Float64,2}:
 -1.40748     0.0744596   0.173258    0.616121   …  -0.60472       0.196305 
 -0.695838   -1.32505    -2.11177     0.294728      -0.393958     -0.598907 
  1.4993     -0.0584219   0.593892   -0.456492      -0.105092      1.76475  
 -1.62447     0.417836    0.289153    0.0588688     -1.16179      -0.114897 
 -0.860024   -2.09391    -0.732752   -2.91457       -0.937152     -0.418518 
 -2.63204    -1.29248    -1.31736    -0.272196   …  -1.20846      -1.43494  
 -0.334512   -1.5214     -2.36533    -0.172815      -0.960779     -2.53669  
  1.07769     0.758103    0.229095    0.924045       1.12853      -1.47241  
  0.852962   -0.737543    0.35169    -1.85           1.15123      -0.319026 
  1.28303     1.5291      1.13272     1.66387        1.61024       0.0514838
  1.01804     0.134116    0.258706   -0.297892   …   0.443519      0.69363  
  0.157477    0.375434    0.433345    1.02332       -0.374164      0.200718 
  0.0106661  -0.238761   -0.216038   -0.488643     

### only use datas with inclcode==1 

In [17]:
inclcode = readcsv("data/inclcode.csv")
data_standardized = data_standardized'[vec(inclcode.==1), :]'

222×58 Array{Float64,2}:
  0.173258    0.616121   -0.449517    …  -0.60472       0.196305 
 -2.11177     0.294728   -0.119795       -0.393958     -0.598907 
  0.593892   -0.456492   -0.632765       -0.105092      1.76475  
  0.289153    0.0588688   0.537598       -1.16179      -0.114897 
 -0.732752   -2.91457    -1.79397        -0.937152     -0.418518 
 -1.31736    -0.272196   -0.875153    …  -1.20846      -1.43494  
 -2.36533    -0.172815   -0.148152       -0.960779     -2.53669  
  0.229095    0.924045    0.891152        1.12853      -1.47241  
  0.35169    -1.85       -0.861188        1.15123      -0.319026 
  1.13272     1.66387     0.919308        1.61024       0.0514838
  0.258706   -0.297892    0.191752    …   0.443519      0.69363  
  0.433345    1.02332    -0.40217        -0.374164      0.200718 
 -0.216038   -0.488643    0.124493       -0.567269      0.420373 
  ⋮                                   ⋱                          
  0.427577   -0.21763    -0.00601165  …   0.0423418

## Compute Total Sum of Squares

In [18]:
tss = sum(nansum(DataFrame(data_standardized.^2)))

12700.0

## Remove columns containing NaN

In [19]:
xbal, _ = dropna_col(data_standardized)

([0.173258 0.616121 … 0.258725 -0.60472; -2.11177 0.294728 … -1.11863 -0.393958; … ; 0.321295 0.265555 … 0.707057 0.925641; 0.074715 1.18534 … 0.555908 0.814873], Bool[true true … true false])

# Least square

In [20]:
nt, ns = size(data_standardized)
nfac_o = 0 # number of observed factors
nfac_u = 1 # number of unobserved factors
nfac_t = nfac_o + nfac_u # total number of factors

1

## Obtain initial value by PCA

In [21]:
function estimate!(xbal, xdatastd, data_standardized, data,
                  initperiod, lastperiod, 
                  nfac_u, nfac_o, nfac_t, tol)#,
                  #::LeastSquare, ::PCA)
    # Get initial F_t given Lambda_t using PCA
    f = pca_score(xbal, nfac_u)
    ssr = 0
    diff = 1000
    while diff > tol*nt*ns
        ssr_old = ssr
        # given F_t, get Lambda_t
        if size(data_standardized, 1) >= nt_min # if sample size is enough
            lambda = nanols(data_standardized, f)[1]'
        end
        # given Lambda_t, get F_t
        # by regressing X_t on Lambda_t for each t
        tmp = nanols(data_standardized', lambda[:, nfac_o+1:end])
        b, ehat = tmp[1]', tmp[2]
        b = tmp[1]'
        f = b
#         ehat = data_standardized - f*lambda'
        ssr = sum(nansum(ehat.^2))
        diff = abs(ssr_old - ssr)
        println(diff)
    end

    lambda = lambda.*xdatastd

    fac_est = fill(NaN, size(data, 1), nfac_t)
    fac_est[initperiod:end,  :] = f

    return fac_est, lambda
end

estimate! (generic function with 1 method)

In [22]:
# Get F_t given Lambda_t
f = pca_score(xbal, nfac_u)
fa = f

222×1 Array{Float64,2}:
  3.59133  
  3.81922  
 -4.8285   
  3.57713  
  7.32604  
  9.797    
  8.64339  
 -0.0988581
 -2.12375  
 -5.12746  
 -2.96291  
 -1.34933  
  1.74576  
  ⋮        
 -3.2693   
 -0.524436 
 -0.440566 
 -1.56502  
 -1.99228  
 -1.82968  
 -2.3149   
 -3.62214  
 -1.37052  
 -4.2118   
 -3.41265  
 -3.06683  

In [23]:
lambda = Matrix{Float64}(ns, nfac_t)
ssr = 0

diff = 1000
tol = 1e-8
while diff > tol*nt*ns
    ssr_old = ssr
    # given F_t, get Lambda_t
    for i = 1:ns
        if size(data_standardized, 1) >= nt_min # if sample size is enough
            xdata_stdi = data_standardized[:, i]
            b, _, _ = nanols(xdata_stdi[:, 1], fa)
            lambda[i, :] = b'
        end
    end

    edata = data_standardized
    # given Lambda_t, get F_t
    for t = 1:nt
        b, _, _ = nanols(edata[t, :], lambda[:, nfac_o+1:end])
        f[t, :] = b'
    end
    fa = f
    ehat = data_standardized - fa*lambda'
    ssr = sum(nansum(ehat.^2))
    diff = abs(ssr_old - ssr)
    println(diff)
end

lambda = lambda.*xdatastd

fac_est = fill(NaN, size(data, 1), nfac_t)
fac_est[initperiod:end,  :] = fa

fac = fac_est

224×1 Array{Float64,2}:
 NaN        
 NaN        
   3.55906  
   3.97141  
  -5.29767  
   3.52022  
   7.35006  
   9.7971   
   8.80436  
   0.0734791
  -2.08731  
  -5.13417  
  -2.95774  
   ⋮        
  -3.28253  
  -0.435026 
  -0.489242 
  -1.55541  
  -2.36165  
  -2.07796  
  -1.93469  
  -3.78706  
  -1.45445  
  -4.59681  
  -3.16031  
  -3.344    

In [24]:
fac_est, lambda = estimate!(xbal, xdatastd, data_standardized, data,
                  initperiod, 1, 
                  nfac_u, nfac_o, nfac_t, 1e-8)

7805.186076916562
0.48167949559046974
0.002725418577028904
4.325549707573373e-5


([NaN; NaN; … ; -3.16031; -3.344], [-0.000891552 -0.000712904 … -0.783265 -0.00115543; -0.000863262 -0.000690283 … -0.758412 -0.00111877; … ; -0.00151168 -0.00120877 … -1.32807 -0.0019591; -0.000940908 -0.000752371 … -0.826627 -0.0012194])

The above code are mostly coming from `factor_estimation_ls.m`


## Part of  `factor_estimation_ls_full.m`

In [25]:
fac = copy(fac_est)

224×1 Array{Float64,2}:
 NaN        
 NaN        
   3.55906  
   3.97141  
  -5.29767  
   3.52022  
   7.35006  
   9.7971   
   8.80436  
   0.0734791
  -2.08731  
  -5.13417  
  -2.95774  
   ⋮        
  -3.28253  
  -0.435026 
  -0.489242 
  -1.55541  
  -2.36165  
  -2.07796  
  -1.93469  
  -3.78706  
  -1.45445  
  -4.59681  
  -3.16031  
  -3.344    

7805.186076916561
0.48167949558956025
0.002725418577028904
4.325549707573373e-5


In [26]:
n_series = size(data, 2)
n_uarlag = 4 # number of arlags for uniqueness

lam_mat = Matrix{Float64}(n_series, nfac_t)
uar_coef_mat = Matrix{Float64}(n_series, n_uarlag)
uar_ser_mat = Vector{Float64}(n_series)
r2_mat = Vector{Float64}(n_series)
for is = 1:n_series
    tmp, numrow = dropna_row([data[initperiod:end, is] fac[initperiod:end, :]])
    if count(numrow) >= nt_min # if available sample size is large enough
        bols, uhat = 
        ols(tmp[:, 1], [tmp[:, 2:end] ones(count(numrow))])
        y_used = data[initperiod:end, is][numrow, :]
        b = copy(bols)
        lam_mat[is, :] = b[1:end-1]
        ssr = sum(uhat.^2)
        ym = y_used - mean(y_used)
        tss = sum(ym.^2)
        r2_mat[is] = 1-ssr/tss
        if r2_mat[is] < 0.9999
            arcoef, ser = uar(uhat, n_uarlag)
        else
            arcoef, ser = zeros(n_uarlag, 1), 0.0
        end
    end
    uar_coef_mat[is, :] = arcoef'
    uar_ser_mat[is, 1] = ser
end

# `varest.m`

In [27]:
function estimate_var(y, nlag::Integer, initperiod::Integer)
    
    T, ns = size(y, 1), size(y, 2)

    x = [ones(T, 1) lagmat(y, 1:nlag)]
    trend = initperiod:T
    
    y = y[initperiod:T, :]
    x = x[initperiod:T, :]
    
    betahat, ehat, numrow = nanols(y, [x trend])
    numrow = vec(numrow)
    y_used, x_used = y[numrow, :], [x trend][numrow, :]

    ndf = size(x_used, 1) - size(x_used, 2) # degree of freedom (T-K)
    seps = ehat[numrow]'*ehat[numrow]/ndf# does not work with multiple dimensions, so need to fix

    resid = fill(NaN, T, ns)
#     resid[Int.(x_used[:, end]), :] = ehat
    resid = ehat

    b = betahat[2:end, :]'
    const_coef = betahat[1, :]

    M = zeros(size(b, 2), size(b, 2))
    M[1:size(b, 1), :] = b

    if size(b, 2) > size(b, 1)
        M[size(b, 1)+1:end, 1:end-size(b, 1)] =
            eye(size(b, 2)-size(b, 1))
    end

    Q = zeros(ns, size(M, 1))
    Q[1:ns, 1:ns] = eye(ns)
    G = zeros(size(M, 1), ns)
    G[1:ns, 1:ns] = (chol(seps))'
    return M, Q, G
end

estimate_var (generic function with 1 method)

In [28]:
M, Q, G = estimate_var(fac, 4, initperiod)

([0.898542 -0.195667 … -0.13101 0.000909197; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0], [1.0 0.0 … 0.0 0.0], [2.83044; 0.0; … ; 0.0; 0.0])

`varest.m` part is done.

In [29]:
"""
VAR model can be expressed in state-space form:
```math
y_t = Qz_t
```
```
z_t = Mz_{t-1}+Gu_t
```
where ``z_t`` has ``y_t`` and lagged ``y_t``s
"""
struct VARModel{TA <: AbstractArray, TM <: AbstractMatrix}
    y::TA
    nlag::Int
    withconst::Bool
    initperiod::Int
    lastperiod::Int
    T::Int
    ns::Int
    resid::TA
    betahat::TA
    M::TM
    Q::TM
    G::TM
    seps::TM
end
function VARModel(y::AbstractArray, nlag::Integer=1; 
                  withconst::Bool=true,
                  initperiod::Integer=1, lastperiod::Integer=size(y, 1))
    T, ns = size(y, 1), size(y, 2)
    resid = fill(NaN, size(y))
    betahat = fill(NaN, nlag+1+withconst, ns)
    M = fill(NaN, ns*nlag+1, ns*nlag+1)
    Q = fill(NaN, ns, ns*nlag+1)
    G = fill(NaN, ns*nlag+1, ns)
    seps = fill(NaN, ns, ns)
    return VARModel(y, nlag, withconst, initperiod, lastperiod, T, ns, resid, betahat, M, Q, G, seps)
end
function estimate_var!(varm::VARModel, compute_matrices::Bool=true)
    initperiod, lastperiod = varm.initperiod, varm.lastperiod
    withconst, nlag = varm.withconst, varm.nlag
    resid, seps = varm.resid, varm.seps
    
    y_restricted = varm.y[initperiod:lastperiod, :]
    trend = initperiod:lastperiod
    
    # regressors
    withconst || (x = lagmat(y_restricted, 1:nlag))
    !withconst || (x = [ones(lastperiod-initperiod+1) lagmat(y_restricted, 1:nlag)])
    regressors = [x trend]
    
    # do OLS ignoring the samples containing NaN
    betahat, ehat, numrows = nanols(y_restricted, regressors)
    varm.betahat .= betahat
    
    T_used = count(numrows) # used sample size
    K = size(regressors, 2) # number of regressors
    
    ndf = T_used - K # degree of freedom (T-K)
    seps .= ehat[numrows]'*ehat[numrows]/ndf # covariance matrix of error term
                                             # does not work with multiple dimensions, so need to fix
    resid[initperiod:lastperiod, :] .= ehat
    
    !compute_matrices || fill_matrices!(varm, betahat)
    return nothing
end
function fill_matrices!(varm::VARModel, betahat::Array)
    ns, nlag = varm.ns, varm.nlag
    M, Q, G = varm.M, varm.Q, varm.G
    
    b = betahat[2:end, :]' # now, each row corresponds to each equation

    M .= zeros(ns*nlag+1, ns*nlag+1)
    M[1:ns, :] .= b # coefficients of VAR
    M[ns+1:end, 1:end-ns] .= eye(ns*nlag+1-ns)　# lag part

    Q .= zeros(ns, ns*nlag+1)
    Q[1:ns, 1:ns] .= eye(ns)
    G .= zeros(ns*nlag+1, ns)
    G[1:ns, 1:ns] .= (chol(varm.seps))'
    return nothing
end

fill_matrices! (generic function with 1 method)

In [30]:
varm = VARModel(fac, 4, initperiod= initperiod)
estimate_var!(varm)

In [31]:
varm.M

5×5 Array{Float64,2}:
 0.898542  -0.195667  0.105248  -0.13101  0.000909197
 1.0        0.0       0.0        0.0      0.0        
 0.0        1.0       0.0        0.0      0.0        
 0.0        0.0       1.0        0.0      0.0        
 0.0        0.0       0.0        1.0      0.0        

# Plotting

In [32]:
j = 1
yf = lam_mat[j, :]' * fac'

y4f = sum(lagmat(yf', 0:3), 2)
y4 = sum(lagmat(data[:, 1], 0:3), 2)

224×1 Array{Float64,2}:
 NaN          
 NaN          
 NaN          
 NaN          
   0.00902449 
  -0.0186939  
  -0.0141709  
  -0.0301671  
  -0.0453175  
  -0.0229932  
  -0.00884106 
   0.0235039  
   0.0346782  
   ⋮          
   0.00394735 
   0.000831648
   0.0049777  
  -0.00596135 
  -0.00466148 
  -0.00421219 
   0.000788617
   0.00933866 
  -0.00264566 
   0.0043127  
   0.00549259 
   0.00242789 

In [ ]:
plot(year, [100*y4 100*y4f], lw=2, lab=["GDP" "Common Component"])